In [2]:
# 数据分组学习目标
# 1、应用groupby进行分组，并对分组数据进行聚合、转换、过滤
# 2、应用自定义函数处理分组之后的数据

In [2]:
# 导入Pandas包
import pandas as pd

# 导入numpy包
import numpy as np

In [12]:
df = pd.read_csv('../data/gapminder.tsv', sep='\t')
df

In [5]:
# 对数据按照年份分组，并求出生活成本平均值
df.groupby('year')['lifeExp'].mean()

In [7]:
# 返回本列唯一值
years = df.year.unique()
years

In [8]:
# 针对1952年的数据取子集
y1952 = df.loc[df.year==1952,:]
y1952


In [9]:
y1952.lifeExp.mean()


In [11]:
# Pandas内置的聚合方法

# Pandas方法	Numpy函数	        说明
# count	        np.count_nonzero	频率统计(不包含NaN值)
# size		                        频率统计(包含NaN值)
# mean	        np.mean	            求平均值
# std	        np.std	            标准差
# min	        np.min	            最小值
# quantile()	np.percentile()	    分位数
# max	        np.max	            求最大值
# sum	        np.sum	            求和
# var	        np.var	            方差
# describe		                    计数、平均值、标准差，最小值、分位数、最大值
# first		                        返回第一行
# last		                        返回最后一行
# nth		                        返回第N行(Python从0开始计数)


In [12]:
df.groupby('continent').lifeExp.describe()


In [13]:
# 聚合函数
df.groupby('continent').lifeExp.agg(np.mean)

In [14]:
# agg和 aggregate效果一样
df.groupby('continent').lifeExp.aggregate(np.mean)

In [25]:
def my_mean(values):
    """
    计算平均值
    :param values: 
    :return: 
    """
    n = len(values)
    s = np.sum(values)
    return s / n

In [26]:
df.groupby('year').lifeExp.agg(my_mean)

In [27]:
def my_mean_diff(values, diff):

    """
    计算平均值和diff_value之差
    :param values: 
    :param diff: 
    :return: 
    """
    n = len(values)
    s = np.sum(values)
    mean = s / n
    return mean - diff

In [30]:
# 计算整个数据集的平均年龄
global_mean = df.lifeExp.mean()
# 调用自定义函数 计算平均值的差值
df.groupby('year').lifeExp.agg(my_mean_diff, diff = global_mean)

In [32]:
# 同时传入多个函数
# 按年计算lifeExp 的非零个数,平均值和标准差
df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])


In [34]:
# 向agg/aggregate中传入字典
df.groupby('year').agg({'lifeExp':'mean','pop':'median','gdpPercap':'median'})


In [36]:
column_fun = {'lifeExp':'mean','pop':'median','gdpPercap':'median'}
columns = {'lifeExp':'平均寿命','pop':'人口','gdpPercap':'人均GDP'}
df.groupby('year').agg(column_fun).rename(columns = columns).reset_index()

In [38]:
# 转换
# 1、transform 转换，需要把DataFrame中的值传递给一个函数， 而后由该函数"转换"数据。
# 2、aggregate(聚合) 返回单个聚合值，但transform 不会减少数据量

In [70]:
# transform分组填充缺失值
tips_10 = pd.read_csv('../data/tips_10.csv.csv')
tips_10


In [5]:
# 查看缺失情况
count_sex = tips_10.groupby('sex').count()
count_sex


In [1]:
def fill_na_mean(x):
    """
    求完平均值，函数填充缺失值
    :param x: 
    :return: 
    """
    avg = x.mean()
    return x.fillna(avg)

In [7]:
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
total_bill_group_mean

In [9]:
# 将计算的结果赋值新列
tips_10['fill_total_bill'] = total_bill_group_mean
tips_10

In [10]:
# 使用transform分组计算z分数
def my_zscore(x):
    """
    计算z-score = x - 平均值/标准差
    :param x: 
    :return: 
    """
    return (x - x.mean()) / x.std()

In [14]:
print(df.shape)
df.groupby('year').lifeExp.transform(my_zscore)

In [15]:
# 过滤
tips = pd.read_csv('../data/tips.csv')
tips

In [19]:
# 查看用餐人数
tips['size'].value_counts()

In [25]:
# 将分组之后，小于30的数据过滤掉
filtered_tips = tips.groupby('size').filter(lambda x : x['size'].count() > 30)
filtered_tips

In [53]:
filtered_tips['size'].value_counts()

In [52]:
# DataFrameGroupBy对象
print(type(tips.groupby('size')))


In [51]:
# 根据性别分组
tips_group_by_sex = tips_10.groupby('sex')
tips_group_by_sex

In [34]:
# 查看计算过的分组数据
tips_group_by_sex.groups

In [55]:
# 获取分组后的均值
# pd.read_csv('../data/tips_10.csv').groupby('sex').mean()
grouped = pd.read_csv('../data/tips_10.csv')[['sex', 'total_bill', 'tip', 'size']].groupby('sex')
grouped

In [56]:
grouped.mean()

In [57]:
grouped.get_group('Female')

In [63]:
# 遍历分组
traversal = [sex_group for sex_group in grouped]
traversal

In [65]:
# DataFrameGroupBy对象直接传入索引，会报错
grouped[0]

In [68]:
sex_group = traversal[0]

# 查看数据类
print(type(sex_group))

# 查看元素个数
print(len(sex_group))

# 查看第一个元素
print(sex_group[0])

# 查看第一个元素数据类型
print(type(sex_group[0]))

# 查看第二个元素
print(sex_group[1])

# 查看第二个元素数据类型
print(type(sex_group[1]))

In [77]:
# 多个分组
multi_gp_tips = pd.read_csv('../data/tips_10.csv')[['sex', 'total_bill', 'tip', 'size', 'time']]
multi_gp_tips

In [78]:
group_avg = multi_gp_tips.groupby(['sex', 'time']).mean()
group_avg

In [81]:
# 分别查看分组之后结果的列名和行索引
print(group_avg.columns)
print(group_avg.index)

In [83]:
# 多个分组之后返回的是MultiIndex，如果想得到一个普通的DataFrame，可以在结果上调用reset_index方法
group_avg.reset_index()

In [85]:
# 也可以在分组的时候通过as_index = False参数（默认是True），效果与调用reset_index()一样
multi_gp_tips.groupby(['sex','time'],as_index = False).mean()

In [86]:
# 总结
# 1、分组是数据分析中常见的操作，有助于从不同角度观察数据
# 2、分组之后可以得到DataFrameGroupby对象，该对象可以进行聚合、转换、过滤操作
# 3、分组之后的数据处理可以使用已有的内置函数，也可以使用自定义函数
# 4、分组不但可以对单个字段进行分组，也可以对多个字段进行分组，多个字段分组之后可以得到MultiIndex数据，可以通过reset_index方法将数据变成普通的DataFrame